In [1]:
#import libraries
import requests
from datetime import datetime as dt
from datetime import timedelta
import pandas as pd


In [2]:
# get events from n days ago
iso = 562 #Look in ACLED documentation ISO code for number this is for Niger
limit = 400

api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}'.format(iso)
print (api_url, type(api_url)) #creates request according to ACLED format specifications - p. 13

https://api.acleddata.com/acled/read?terms=accept&iso=562 <class 'str'>


In [3]:
response = requests.get(api_url)
data = response.json()
data.keys()
data['count'] 

500

### From the documentation we know this is the max return --- How can we get all the results?

In [4]:
# Let's mkae a function that updates our search to get the new pages

def ping_acled(api_url): 
    '''
    Takes one parameter search term for API
    '''
    
    response = requests.get(api_url)
    data = response.json()
    return data
    

In [5]:

results = [] # empty data strcture to store results
num_results = 500 # condition to continue adding pages
count = 0  # tracker of results
page = 1 #Per the documentation each page will give us more results

while num_results == 500: #if less 500 or 0 we know we have all the results
    print ("starting ", page, " ", num_results) #just to see our progress
    api_url = 'https://api.acleddata.com/acled/read?terms=accept&iso={}&page={}'.format(iso,page) #the search
    data = ping_acled(api_url) #call the previous function 
    results.append(data['data']) #store in our results
    count += data['count'] #Track number of results
    num_results = data['count'] #update our condition
    page += 1 #update our page variable
    print ("Total Results ", count) #Track our progress
    



starting  1   500
Total Results  500
starting  2   500
Total Results  1000
starting  3   500
Total Results  1408


In [6]:
#Now I want to put them together into one giant result
super_list = []
for res in results: 
    super_list += res
    print (len(super_list))

500
1000
1408


In [7]:
#convert it into an pandas data frame or just use your data structure and do more stuff
niger_res = pd.DataFrame(super_list)
niger_res.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,...,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,7233612,562,NIR1460,1460,2020-09-23,2020,1,Battles,Armed clash,Boko Haram - Jamatu Ahli is-Sunnah lid-Dawatai...,...,Alari,14.4664,13.4416,1,Jeunesse Diffa,New media,"On 23 September 2020, presumed Boko Haram (JAS...",7,1601324148,NER
1,7233305,562,NIR1459,1459,2020-09-22,2020,1,Strategic developments,Looting/property destruction,Unidentified Armed Group (Niger),...,Taroum,14.6744,2.2729,2,Undisclosed Source; Whatsapp,Local partner-Other,"On 22 September 2020, unknown gunmen aboard mo...",0,1601324148,NER
2,7233937,562,NIR1458,1458,2020-09-22,2020,1,Violence against civilians,Attack,Unidentified Armed Group (Niger),...,Kossa,14.6238,0.2657,1,Whatsapp,New media,"On 22 September 2020, two unknown gunmen on a ...",1,1601324149,NER
3,7233304,562,NIR1457,1457,2020-09-21,2020,1,Strategic developments,Looting/property destruction,Unidentified Armed Group (Niger),...,Zigui Zeno,15.0237,2.4502,2,L'Evenement Niger; Tadress24info; Urgence Till...,New media-National,"On 21 September 2020, unknown gunmen seized an...",0,1601324147,NER
4,7233611,562,NIR1456,1456,2020-09-21,2020,1,Protests,Peaceful protest,Protesters (Niger),...,Diffa,13.3200,12.6100,1,Jeunesse Diffa,New media,"On 21 September 2020, a number of students sta...",0,1601324148,NER


### Do the right thing, take some time to look at the codebook and see what these columns are

In [8]:
niger_res.columns


Index(['data_id', 'iso', 'event_id_cnty', 'event_id_no_cnty', 'event_date',
       'year', 'time_precision', 'event_type', 'sub_event_type', 'actor1',
       'assoc_actor_1', 'inter1', 'actor2', 'assoc_actor_2', 'inter2',
       'interaction', 'region', 'country', 'admin1', 'admin2', 'admin3',
       'location', 'latitude', 'longitude', 'geo_precision', 'source',
       'source_scale', 'notes', 'fatalities', 'timestamp', 'iso3'],
      dtype='object')

### Homework --- Make a map of some ACLED Data (absolutely use the code from the Global Terrorism Database exercise)

In [9]:
#Imports necessary aspects of Bokeh for plotting on a map
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Transformer
from bokeh.plotting import figure,output_notebook,show
output_notebook()
tile_provider = get_provider('STAMEN_TERRAIN')

Loading BokehJS ...

In [10]:
# Take the data reduced to a country and get the lat/long of the attacks and the name of the group
country_map = niger_res[['latitude', 'longitude']]
#see the data this time first 7 rows
#country_map.head()
print(country_map)
type(niger_res['latitude'])
#niger_res['latitude'][0]
#float(niger_res['latitude'])
#float(niger_res['longitude'])

     latitude longitude
0     14.4664   13.4416
1     14.6744    2.2729
2     14.6238    0.2657
3     15.0237    2.4502
4     13.3200   12.6100
...       ...       ...
1403  21.9400   13.6400
1404  13.5200    2.1200
1405  13.3200   12.6100
1406  13.5200    2.1200
1407  13.5200    2.1200

[1408 rows x 2 columns]


pandas.core.series.Series

In [11]:
Niger_x = niger_res['latitude']
Niger_y = niger_res['longitude']
points_gps = [(Niger_x , Niger_y)]
print(points_gps)


[(0       14.4664
1       14.6744
2       14.6238
3       15.0237
4       13.3200
         ...   
1403    21.9400
1404    13.5200
1405    13.3200
1406    13.5200
1407    13.5200
Name: latitude, Length: 1408, dtype: object, 0       13.4416
1        2.2729
2        0.2657
3        2.4502
4       12.6100
         ...   
1403    13.6400
1404     2.1200
1405    12.6100
1406     2.1200
1407     2.1200
Name: longitude, Length: 1408, dtype: object)]


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
               

In [13]:
#create pyproj transformer to convert form lat/long to web mercator
transformer = Transformer.from_crs('epsg:4326','epsg:3857')

In [14]:
pts = [(11.00,0.00), (23.00, 15.90)]
bbox = []
bbox = []
for pt in transformer.itransform(pts): 
    bbox.append(pt)

In [15]:
type(Niger_x)

pandas.core.series.Series

In [16]:
Niger_x = list(Niger_x)
Niger_y = list(Niger_y)

In [17]:
Niger_x

['14.4664',
 '14.6744',
 '14.6238',
 '15.0237',
 '13.3200',
 '13.5316',
 '13.6987',
 '13.4671',
 '13.8986',
 '13.5337',
 '13.5997',
 '14.2495',
 '13.4580',
 '13.3661',
 '13.6933',
 '13.6933',
 '14.0493',
 '14.0740',
 '14.7702',
 '19.3667',
 '20.5575',
 '13.4580',
 '13.8986',
 '13.5084',
 '13.2668',
 '12.9486',
 '14.2495',
 '13.3151',
 '13.9223',
 '13.1576',
 '13.4934',
 '13.2909',
 '13.4210',
 '13.2806',
 '22.6532',
 '14.0939',
 '13.2052',
 '13.5200',
 '13.3200',
 '13.4497',
 '15.6207',
 '12.8500',
 '14.2495',
 '16.7281',
 '14.2117',
 '13.3253',
 '13.9656',
 '13.4210',
 '12.1779',
 '13.8914',
 '14.3999',
 '14.7326',
 '14.2614',
 '13.9768',
 '13.7838',
 '13.6458',
 '15.0744',
 '14.1937',
 '13.3013',
 '13.3013',
 '13.3200',
 '14.0939',
 '14.3397',
 '13.3616',
 '13.3200',
 '14.2495',
 '16.8702',
 '14.2495',
 '14.3066',
 '15.6853',
 '13.3658',
 '13.0979',
 '13.4497',
 '13.7812',
 '14.7326',
 '14.0397',
 '13.7379',
 '13.6669',
 '14.7348',
 '15.7709',
 '13.8083',
 '14.9523',
 '14.5482',
 '13

In [18]:
for idx in range(len(Niger_x)):
    Niger_x[idx], Niger_y[idx] = transformer.transform(float(Niger_x[idx]), float(Niger_y[idx]))
        

In [19]:
list(Niger_x)

[1496312.0674468658,
 253018.07062403148,
 29577.588703772784,
 272755.0163416789,
 1403738.7789031796,
 1417197.3053400863,
 1481606.7627130747,
 1427082.476122529,
 1464341.109691038,
 1426904.3649372598,
 721762.1824563479,
 1459309.468707182,
 744794.1851014761,
 1417932.0139793223,
 1439104.9811282028,
 1439104.9811282028,
 1446852.8176874144,
 904849.3489640448,
 123041.43317380527,
 645653.0466009867,
 1507299.301188162,
 744794.1851014761,
 1464341.109691038,
 733628.8401749108,
 114480.96433180255,
 390230.4749758205,
 1459309.468707182,
 1397983.5612291675,
 1461324.3514905402,
 1388009.3348540901,
 1412533.0186758484,
 1368595.2156597432,
 1376610.2189968592,
 1369652.7508222796,
 1568580.6808698592,
 1481896.193389137,
 1382510.1520089023,
 235997.32048173997,
 1403738.7789031796,
 1425279.1003716781,
 532463.3883623861,
 165120.20069366271,
 1459309.468707182,
 829241.1508172535,
 161758.35207170583,
 288740.49521959294,
 631782.6380481449,
 1376610.2189968592,
 346103.428

In [21]:
#Plots the bounding box
p = figure(x_range=(bbox[0][0], bbox[1][0]),y_range=(bbox[0][1], bbox[1][1]),x_axis_type="mercator", y_axis_type="mercator")
#add the map form the Bokeh map vendor in this case Stamen_Terrain --- see documentation
p.add_tile(tile_provider)
# Places a circle for each converted lat/long attack 

p.circle(x=Niger_x, y=Niger_y, size=5, color= "firebrick")
#shows the plot
show(p)